# Electric Utilities and Wildfires in California

Last week, the California Legislature released a 90-page audit of the California Public Utilities Commission, the entity responsible for overseeing utility companies like Pacific Gas & Electric (PG&E). This project will visualize some of the report's key findings in an interactive scrollytelling format.

Report: https://www.auditor.ca.gov/pdfs/reports/2021-117.pdf

### Mapping Fires Started by Electrical Equipment

The starting point map will be a version of [this map](http://www.auditor.ca.gov/reports/2021-117/supplemental-fire-incident.html) produced by the auditors. It's sourced from incident data published on [CPUC's website](https://www.cpuc.ca.gov/industries-and-topics/wildfires). These data are published in PDFs which need to be parsed.

In [1]:
#!brew install ghostscript tcl-tk
#!pip install "camelot-py[base]"
#!pip install opencv-python
import camelot
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
filenames = ['PGE Fire Incident  Report 2020.pdf',
             '2014-2019_Consolidated_PGE_Annual_Fire_Report_20220328.xlsx',
            'SCE-Fire-Incident-Data-2014-2019.pdf',
            'SDGE Fire Incident Report 2020.pdf',
            'SDGE-Fire-Incident-Data-2014-2019.pdf',
            'SCE Fire Report 2020.pdf'] #not recognized as a table right now, need to revisit
    

In [33]:
fire_tables = []
for file in filenames:
    print(file)
    if '.xlsx' in file:
        fires = pd.read_excel('data/' + file, skiprows=1)
    elif 'SCE Fire Report' in file:
        tables = camelot.read_pdf('data/' + file, flavor='stream')
        fires = tables[0].df
        fires.columns = fires.iloc[0]
        fires = fires[1:]
        table_dateyear = fires['Date\nYear'].str.split('\n', expand=True)
        table_dateyear.columns = ['Date', 'Year']
        table_end = fires['Suspected Initiating Event\nEquipment /Facility Failure\nContact From Object\nFacility Contacted\nContributing Factor'].str.split('\n', expand=True)
        table_end.columns = ['Suspected Initiating Event', 'Contact From Object', 'Facility Contacted/Contributing Factor']
        fires = fires.join(table_end).join(table_dateyear)
    else:
        tables = camelot.read_pdf('data/' + file)
        fires = tables[0].df
        if '2019' in file:
            fires.columns = fires.iloc[2]
            fires = fires[3:]
        else: 
            fires.columns = fires.iloc[1]
            fires = fires[2:]
        fires.reset_index(inplace=True, drop=True)
    fires.columns = fires.columns.str.lower().str.replace('\n', '').str.replace(' ', '_')
    fire_tables.append(fires)

PGE Fire Incident  Report 2020.pdf
2014-2019_Consolidated_PGE_Annual_Fire_Report_20220328.xlsx
SCE-Fire-Incident-Data-2014-2019.pdf
SDGE Fire Incident Report 2020.pdf
SDGE-Fire-Incident-Data-2014-2019.pdf
SCE Fire Report 2020.pdf


#### Cleaning the tables

In [49]:
fire_tables[2].material_at_origin = fire_tables[2].longitude.str.split(' ', expand=True).iloc[:, [1]]
fire_tables[2].longitude = fire_tables[2].longitude.str.split(' ', expand=True).iloc[:, [0]]

In [50]:
fire_tables[4].material_at_origin = fire_tables[4].longitude.str.split(' ', expand=True).iloc[:, [1]]
fire_tables[4].longitude = fire_tables[4].longitude.str.split(' ', expand=True).iloc[:, [0]]

In [51]:
fire_tables[5].rename(columns={'':'longitude', 'longitude_material_at_origin':'material_at_origin'}, inplace=True)

In [52]:
#Manually adding in the values for type because they are almost all overhead, simpler than string split
fire_tables[5]['type'] = 'Overhead'
fire_tables[5].loc[fire_tables[5].circuitfipa_form_name == 'Heers\nVIPA_2020_1110_1116', 'type'] = 'Underground'

In [59]:
fire_tables[5]['size'] = fire_tables[5].suppressed_by.str.split('\n', expand=True).iloc[:,[0]]

In [53]:
fire_tables[0].replace('N.A.', np.nan) #replace NA with actual null
fire_tables[3].rename(columns={'suspected_ignition_cause': 'suspected_initiating_event'}, inplace=True)

In [54]:
fire_tables[3]['utility_name'] = 'SDG&E'

In [55]:
fire_tables[0].rename(columns={'': 'utility_name'}, inplace=True)
fire_tables[2].rename(columns={'column1': 'utility_name'}, inplace=True)

In [60]:
clean_tables = []
for table in fire_tables:
    clean = table[['utility_name', 'date', 'time', 'latitude', 'longitude', 'material_at_origin',\
                  'size', 'type', 'suspected_initiating_event']]
    if len(clean.columns) == 11:
        clean.columns = ['utility_name', 'date', 'date2', 'time', 'time2', 'latitude', 'longitude',\
                         'material_at_origin', 'size', 'type', 'suspected_initiating_event']
        clean.drop(columns=['date2', 'time2'], inplace=True)
    clean_tables.append(clean)
    

/Users/jmingram/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [61]:
all_fires = pd.concat(clean_tables)

In [62]:
all_fires = all_fires.replace('Not Provided', np.nan).replace('Not Provided -118.72947', np.nan).\
            replace('Not Provided Not Provided', np.nan).replace('Not ProvideNot Provided Not Provided', np.nan).\
            replace('13:04:00 P 33.3391670', '33.3391670').replace('', np.nan).replace('######', np.nan)


In [71]:
all_fires.to_csv('electrical_fires.csv', index=False) 
# manually fix date formats and other abnormalities in excel, fastest way

In [72]:
all_fires = pd.read_csv('electrical_fires.csv')

In [73]:
all_fires.latitude = all_fires.latitude.astype(float)
all_fires.longitude = all_fires.longitude.astype(float)

In [75]:
all_fires.date = pd.to_datetime(all_fires.date, errors='coerce')

In [76]:
all_fires.loc[(all_fires.type == 'Overhead') & (all_fires.suspected_initiating_event == 'Contact From Object'),\
             'highlight'] = True

In [77]:
all_fires.highlight = all_fires.highlight.fillna(False)

In [198]:
all_fires[all_fires.year>=2015].highlight.value_counts(normalize=True)

False    0.578947
True     0.421053
Name: highlight, dtype: float64

In [80]:
all_fires.to_csv('electrical_fires.csv', index=False)

### Run code from here

In [211]:
all_fires = pd.read_csv('qgis/electrical_fires.csv')

In [212]:
all_fires.date = pd.to_datetime(all_fires.date)

In [213]:
all_fires['year'] = all_fires.date.dt.year

In [142]:
all_fires['size'].value_counts()

< 0.25 Acres         2160
.26 - 9.99 Acres      563
10 - 99 Acres          97
Structure Only         78
>5,000                 13
100 - 299 Acres        11
1000 - 4999 Acres       9
300 - 999 Acres         9
100+ Acres              5
Unknown                 5
>5,500                  2
Other                   1
Name: size, dtype: int64

In [141]:
all_fires['size'] = all_fires['size'].str.strip()
all_fires['size'] = all_fires['size'].str.replace('Less Than .25 Acres', '< 0.25 Acres')\
                    .str.replace('<0.25', '< 0.25 Acres').str.replace('3 meters - 0.25 Acres', '< 0.25 Acres')\
                    .str.replace('< 3 meters', '< 0.25 Acres').str.replace('1 meter - < 0.25 Acres', '< 0.25 Acres')\
                    .str.replace('< 3 Meters', '< 0.25 Acres').str.replace('< 1 meter', '< 0.25 Acres')\
                    .str.replace('0.25 - 10 Acres', '.26 - 9.99 Acres').replace('10 - 100 Acres','10 - 99 Acres')\
                    .str.replace('.26 - 9.99 AcresUnknown', '.26 - 9.99 Acres').str.replace('.26 - 9.99 AcresUtility', '.26 - 9.99 Acres')\
                    .str.replace('10 - 99 AcresFire Agency', '10 - 99 Acres').str.replace('10 - 100 Acres', '10 - 99 Acres')\
                    .str.replace('0.26-9.99', '.26 - 9.99 Acres').str.replace('1,000-4,999', '1000 - 4999 Acres')\
                    .str.replace('36434', '1000 - 4999 Acres').str.replace('Greater than 5000 Acres', '>5,000')\
                    .str.replace('> 5000 Acres', '>5,500').str.replace('100-300', '100 - 299 Acres')\
                    .str.replace('10 - 99 AcresUnknown', '10 - 99 Acres').str.replace('Structure-only', 'Structure Only')\
                    .str.replace('300-999', '300 - 999 Acres')

/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
/var/folders/tx/ryc062z51z1287rjpvrp4jlc0000gn/T/ipykernel_14806/812776659.py:9: FutureWarning: The default value of regex wi

In [143]:
all_fires.to_csv('qgis/electrical_fires.csv', index=False)
    

In [168]:
tier2_fires = pd.read_csv('qgis/tier2_fires.csv')
tier3_fires = pd.read_csv('qgis/tier3_fires.csv')
notier_fires = pd.read_csv('qgis/notier_fires.csv')

In [187]:
tier2_fires = tier2_fires[tier2_fires.year>=2015]
tier3_fires = tier3_fires[tier3_fires.year>=2015]
notier_fires = notier_fires[notier_fires.year>=2015]

In [188]:
tier2_fires['size'].value_counts(normalize=True)

< 0.25 Acres         0.624319
.26 - 9.99 Acres     0.290381
10 - 99 Acres        0.050817
>5,000               0.010889
Structure Only       0.007260
100 - 299 Acres      0.005445
300 - 999 Acres      0.005445
1000 - 4999 Acres    0.001815
Unknown              0.001815
Other                0.001815
Name: size, dtype: float64

In [190]:
tier3_fires['size'].value_counts(normalize=True)

< 0.25 Acres         0.721854
.26 - 9.99 Acres     0.201987
10 - 99 Acres        0.033113
>5,000               0.016556
100 - 299 Acres      0.006623
1000 - 4999 Acres    0.006623
Structure Only       0.006623
>5,500               0.003311
100+ Acres           0.003311
Name: size, dtype: float64

In [191]:
tier2and3 = pd.concat([tier2_fires, tier3_fires])

In [192]:
tier2and3['size'].value_counts(normalize=True)


< 0.25 Acres         0.658851
.26 - 9.99 Acres     0.259086
10 - 99 Acres        0.044549
>5,000               0.012896
Structure Only       0.007034
100 - 299 Acres      0.005862
1000 - 4999 Acres    0.003517
300 - 999 Acres      0.003517
Unknown              0.001172
Other                0.001172
>5,500               0.001172
100+ Acres           0.001172
Name: size, dtype: float64

In [194]:
notier_fires['size'].value_counts(normalize=True)


< 0.25 Acres         0.763739
.26 - 9.99 Acres     0.159207
Structure Only       0.032295
10 - 99 Acres        0.030595
100 - 299 Acres      0.003399
300 - 999 Acres      0.002266
100+ Acres           0.002266
Unknown              0.002266
1000 - 4999 Acres    0.002266
>5,000               0.001133
>5,500               0.000567
Name: size, dtype: float64

In [196]:
tier2and3['size'].value_counts()


< 0.25 Acres         562
.26 - 9.99 Acres     221
10 - 99 Acres         38
>5,000                11
Structure Only         6
100 - 299 Acres        5
1000 - 4999 Acres      3
300 - 999 Acres        3
Unknown                1
Other                  1
>5,500                 1
100+ Acres             1
Name: size, dtype: int64

In [197]:
notier_fires['size'].value_counts()


< 0.25 Acres         1348
.26 - 9.99 Acres      281
Structure Only         57
10 - 99 Acres          54
100 - 299 Acres         6
300 - 999 Acres         4
100+ Acres              4
Unknown                 4
1000 - 4999 Acres       4
>5,000                  2
>5,500                  1
Name: size, dtype: int64

In [203]:
#Calculating how much more likley fires in risk zones are to be greater than .25 acres using odds ratios
a = 100-65.9#tier fires more than .25 acres
b = 65.9#tier fires less than .25 acres
c = 100-76.4#non tier fires more than .25 acres
d = 76.4#non tier fires less than .25 acres

In [204]:
(a/b) / (c/d)

1.6751369563540035

## Visualization Plan

Each bullet represents a scroll

- Contextualize with a bar chart showing the % of wildfire acres burned by electric utilities (table 1, page 7)
- Overall fire incidents map (see page 21) 
- Highlight the points casued by contact with foreign objects on overhead lines, which suggest bare lines more susceptible to fire 
- Add the layers for high risk and extreme risk


In text can go something about the hardening and PG&E's report